In [21]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [22]:
dim(df_finale)

[1] 1262   74

In [23]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [24]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [25]:
dim(test)
dim(train)

[1] 379 175

[1] 883 175

In [26]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    441     442 

## Model

In [27]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [28]:

model_svmPoly <- train(x=TrainData, y=TrainClasses, 
               method = "svmPoly", scale= FALSE) 

In [29]:

model_svmRadial <- train(x=TrainData, y=TrainClasses, 
               method = "svmRadial", scale= FALSE) 

Warning message in .local(x, ...):
"Variable(s) `' constant. Cannot scale data."


In [30]:

model_svmLinear <- train(x=TrainData, y=TrainClasses, 
               method = "svmLinear", scale= FALSE)

# Results

In [31]:
print(model_svmPoly)
plot(model_svmPoly,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Polynomial Kernel 

883 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 883, 883, 883, 883, 883, 883, ... 
Resampling results across tuning parameters:

  degree  scale  C     Accuracy   Kappa    
  1       0.001  0.25  0.6001183  0.2137583
  1       0.001  0.50  0.6969496  0.3988108
  1       0.001  1.00  0.7234881  0.4491689
  1       0.010  0.25  0.7297012  0.4605814
  1       0.010  0.50  0.7318727  0.4645879
  1       0.010  1.00  0.7315033  0.4634773
  1       0.100  0.25  0.7321305  0.4642739
  1       0.100  0.50  0.7318036  0.4635335
  1       0.100  1.00  0.7328420  0.4653218
  2       0.001  0.25  0.7000462  0.4048724
  2       0.001  0.50  0.7236132  0.4493341
  2       0.001  1.00  0.7289430  0.4592399
  2       0.010  0.25  0.7349462  0.4706422
  2       0.010  0.50  0.7396322  0.4795148
  2       0.010  1.00  0.7399977  0.4799362
  2       0.100  0.25  0.76

In [32]:
print(model_svmRadial)
plot(model_svmRadial,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Radial Basis Function Kernel 

883 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 883, 883, 883, 883, 883, 883, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.7515752  0.5031686
  0.50  0.7532607  0.5065493
  1.00  0.7610945  0.5221251

Tuning parameter 'sigma' was held constant at a value of 0.01914113
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.01914113 and C = 1.


In [33]:
print(model_svmLinear)
plot(model_svmLinear)

Support Vector Machines with Linear Kernel 

883 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 883, 883, 883, 883, 883, 883, ... 
Resampling results:

  Accuracy   Kappa    
  0.7130109  0.4265623

Tuning parameter 'C' was held constant at a value of 1


ERROR: Error in plot.train(model_svmLinear): There are no tuning parameters with more than 1 value.


In [34]:
# collect resamples
results <- resamples(list(Poly=model_svmPoly,  Radial=model_svmRadial, Linear=model_svmLinear))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: Poly, Radial, Linear 
Number of resamples: 25 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.7281250 0.7583082 0.7661538 0.7658869 0.7739938 0.8054711    0
Radial 0.7215190 0.7461300 0.7630769 0.7610945 0.7784431 0.7951070    0
Linear 0.6635514 0.6972477 0.7151899 0.7130109 0.7275542 0.7666667    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.4622991 0.5165667 0.5332225 0.5322093 0.5498387 0.6111542    0
Radial 0.4426808 0.4938272 0.5259070 0.5221251 0.5571244 0.5904630    0
Linear 0.3262855 0.3957217 0.4306979 0.4265623 0.4568939 0.5314747    0


# Prediction

In [35]:
pred_svmPoly <- predict(model_svmPoly,TestData)
pred_svmPoly.prob <- predict(model_svmPoly,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmPoly) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      74      53
   NotFast   58     194
                                          
               Accuracy : 0.7071          
                 95% CI : (0.6585, 0.7525)
    No Information Rate : 0.6517          
    P-Value [Acc > NIR] : 0.01273         
                                          
                  Kappa : 0.3491          
                                          
 Mcnemar's Test P-Value : 0.70420         
                                          
            Sensitivity : 0.5606          
            Specificity : 0.7854          
         Pos Pred Value : 0.5827          
         Neg Pred Value : 0.7698          
             Prevalence : 0.3483          
         Detection Rate : 0.1953          
   Detection Prevalence : 0.3351          
      Balanced Accuracy : 0.6730          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5606061            0.7854251            0.5826772 
      Neg Pred Value            Precision               Recall 
           0.7698413            0.5826772            0.5606061 
                  F1           Prevalence       Detection Rate 
           0.5714286            0.3482850            0.1952507 
Detection Prevalence    Balanced Accuracy 
           0.3350923            0.6730156

In [36]:
pred_svmRadial <- predict(model_svmRadial,TestData)
pred_svmRadial.prob <- predict(model_svmRadial,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmRadial) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      70      57
   NotFast   53     199
                                         
               Accuracy : 0.7098         
                 95% CI : (0.6612, 0.755)
    No Information Rate : 0.6755         
    P-Value [Acc > NIR] : 0.08418        
                                         
                  Kappa : 0.3435         
                                         
 Mcnemar's Test P-Value : 0.77485        
                                         
            Sensitivity : 0.5691         
            Specificity : 0.7773         
         Pos Pred Value : 0.5512         
         Neg Pred Value : 0.7897         
             Prevalence : 0.3245         
         Detection Rate : 0.1847         
   Detection Prevalence : 0.3351         
      Balanced Accuracy : 0.6732         
                                         
       'Positive' Class : fast           
                                   

Sensitivity          Specificity       Pos Pred Value 
           0.5691057            0.7773438            0.5511811 
      Neg Pred Value            Precision               Recall 
           0.7896825            0.5511811            0.5691057 
                  F1           Prevalence       Detection Rate 
           0.5600000            0.3245383            0.1846966 
Detection Prevalence    Balanced Accuracy 
           0.3350923            0.6732247

In [37]:
pred_svmLinear <- predict(model_svmLinear,TestData)
pred_svmLinear.prob <- predict(model_svmLinear,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmLinear) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      69      58
   NotFast   54     198
                                        
               Accuracy : 0.7045        
                 95% CI : (0.6558, 0.75)
    No Information Rate : 0.6755        
    P-Value [Acc > NIR] : 0.1242        
                                        
                  Kappa : 0.3316        
                                        
 Mcnemar's Test P-Value : 0.7768        
                                        
            Sensitivity : 0.5610        
            Specificity : 0.7734        
         Pos Pred Value : 0.5433        
         Neg Pred Value : 0.7857        
             Prevalence : 0.3245        
         Detection Rate : 0.1821        
   Detection Prevalence : 0.3351        
      Balanced Accuracy : 0.6672        
                                        
       'Positive' Class : fast          
                                        

Sensitivity          Specificity       Pos Pred Value 
           0.5609756            0.7734375            0.5433071 
      Neg Pred Value            Precision               Recall 
           0.7857143            0.5433071            0.5609756 
                  F1           Prevalence       Detection Rate 
           0.5520000            0.3245383            0.1820580 
Detection Prevalence    Balanced Accuracy 
           0.3350923            0.6672066

In [38]:
roc_obj <- roc(TestClasses, as.numeric(pred_svmRadial))
plot(roc_obj, main="Curva ROC")


Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [39]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_svmRadial))

Data: as.numeric(pred_svmRadial) in 127 controls (TestClasses fast) < 252 cases (TestClasses NotFast).
Area under the curve: 0.6704

In [40]:
pred_svmRadial.prob

fast,NotFast
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
